# Colab Papermill Driver
## Install and Import
Download and install or upgrade Papermill automatically and import the neccessary packages.

In [1]:
!pip install --upgrade papermill -q

     |████████████████████████████████| 61kB 7.5MB/s 


In [0]:
import papermill as pm
import json
import os
from google.colab import drive as downDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Connect
You must allow both Google Colab Drive and PyDrive access to your Google Drive. This seems like a redundancy but both drive objects have unique methods required for this patch.

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
upDrive = GoogleDrive(gauth)

In [4]:
downDrive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Navigate
Here you will specify the *target_dir* where the notebook you wish to parameterized is located. You should not need to change it if you are following the Medium tutorial.

In [0]:
target_dir = 'drive/My Drive/Colab Notebooks/'
os.chdir(target_dir)

It's always good to verify you are in the correct folder before moving forward. The assertion will return "Wrong Folder!" if Sample.ipynb is not in the current directory.

In [0]:
filenames = !ls
filenames = filenames[0].split(' ')
assert 'Sample.ipynb' in filenames, "Wrong Folder!"

## Select and Modify
Extract the notebook as a JSON and modify it for Papermill compatibility.

In [0]:
target_notebook = 'Sample.ipynb'
notebook_name = target_notebook[:target_notebook.index('.')]
f = open(target_notebook, "r")
j = json.load(f)
f.close()

In [31]:
j['cells'][0]['metadata']['tags'] = ['parameters']
j['cells'][0]['metadata']

{'colab': {},
 'colab_type': 'code',
 'id': 'lpO4HTHgMYru',
 'tags': ['parameters']}

In [32]:
j['metadata']['kernelspec']['language'] = 'python'
j['metadata']['kernelspec']

{'display_name': 'Python 3', 'language': 'python', 'name': 'python3'}

In [0]:
os.chdir('../../..')
pm_notebook_name = notebook_name + "_PM.ipynb"
f = open(pm_notebook_name, "w")
json.dump(j, f)
f.close()

## Upload and Relocate
Using PyDrive, we create a GoogleDriveFile type and specify its mimeType such that it will be interpreted as a Google Colab file when we upload it. Unfortunately it will upload to the base directory of your Google Drive so we relocate it to the target_dir we specified earlier.

In [0]:
file = upDrive.CreateFile({'title':pm_notebook_name, 'mimeType':'application/vnd.google.colaboratory'})
file.SetContentFile(pm_notebook_name)
file.Upload()

#**STOP HERE!**
**In the left pane go to the Files section and press Refresh before running the next line. If you do not refresh, the next cell will fail!**

Delete the Colab server copy

In [0]:
!rm $pm_notebook_name

Move the notebook to the correct Google Drive folder

In [0]:
!cd 'drive/My Drive' && mv $pm_notebook_name 'Colab Notebooks'

## Execute with Papermill
Create a blank Colab type notebook for Papermill to overwrite.


In [0]:
output_notebook_name = notebook_name + '_PM_Output.ipynb'
file = upDrive.CreateFile({'title':output_notebook_name, 'mimeType':'application/vnd.google.colaboratory'})
file.Upload()

#**STOP HERE!**
**In the left pane go to the Files section and press Refresh before running the next line. If you do not refresh, the next cell will fail!**

Move the output notebook into the correct folder

In [0]:
!cd 'drive/My Drive' && mv $output_notebook_name 'Colab Notebooks'

Set custom parameters for the notebook.

In [43]:
parameters = {'test':"Hello World!"}

_ = pm.execute_notebook(
        input_path = target_dir + pm_notebook_name,
        output_path = target_dir + output_notebook_name,
        parameters = parameters
    )